In [1]:
import sys

sys.path.append("../..")
import pandas as pd
from src.data_loaders.rollcall import (
    get_raw_individual_votes, get_raw_party_membership, get_individual_votes_with_party_enriched, get_individual_votes_with_party_enriched
)

In [2]:
df = get_raw_party_membership()
df_dedupe = df[["icpsr", "congress"]].drop_duplicates().sort_values(["icpsr", "congress"])
df["terms_served"] = (df_dedupe.groupby("icpsr")["congress"].cumcount() + 1).to_frame("terms_served")
df["terms_served"].value_counts().head()

terms_served
1.0    12667
2.0     8967
3.0     6452
4.0     4789
5.0     3799
Name: count, dtype: int64

In [3]:
# Confirmed that John Dingell was the longest serving congresssman in the house
df.loc[df["terms_served"] == df["terms_served"].max()].T.loc["bioname"]

47387    DINGELL, John David, Jr.
Name: bioname, dtype: object

In [4]:
house_df = df.loc[df["chamber"] == "House"]
district_count = house_df.groupby(["state_abbrev", "state_icpsr", "congress"])["district_code"].nunique().to_frame('num_reps')
total_district_count = district_count.groupby("congress")["num_reps"].sum().to_frame('num_reps_total')
count_df = district_count.join(total_district_count)

count_df["pct_pop"] = count_df["num_reps"] / count_df["num_reps_total"]
count_df.xs(118, level="congress").sort_values("pct_pop", ascending=False).head()

,,num_reps,num_reps_total,pct_pop
state_abbrev,state_icpsr,,,
CA,71,52,441,0.117914
TX,49,38,441,0.086168
FL,43,28,441,0.063492
NY,13,26,441,0.058957
IL,21,17,441,0.038549


In [5]:
df = df.set_index(["state_icpsr", "congress"]).join(count_df.droplevel('state_abbrev')).reset_index()


In [10]:
df = get_individual_votes_with_party_enriched()
df.head()

Loading cached data from /Users/declannelson/Desktop/columbia/stat5241/stat5241_team_7/src/data/individual_votes_8.parquet


date bill_number  \
congress session chamber rollnumber icpsr                           
101      2.0     0       371        633.0  1990-01-24      hr2712   
                                    1077.0 1990-01-24      hr2712   
                                    1087.0 1990-01-24      hr2712   
                                    2009.0 1990-01-24      hr2712   
                                    2605.0 1990-01-24      hr2712   

                                            vote_passed  vote_type_amend  \
congress session chamber rollnumber icpsr                                  
101      2.0     0       371        633.0             1                0   
                                    1077.0            1                0   
                                    1087.0            1                0   
                                    2009.0            1                0   
                                    2605.0            1                0   

                                            vote_type_cloture  \
congress session chamber rollnumber icpsr                       
101      2.0     0       371        633.0                   0   
                                    1077.0                  0   
                                    1087.0                  0   
                                    2009.0                  0   
                                    2605.0                  0   

                                            vote_type_concur  \
congress session chamber rollnumber icpsr                      
101      2.0     0       371        633.0                  0   
                                    1077.0                 0   
                                    1087.0                 0   
                                    2009.0                 0   
                                    2605.0                 0   

                                            vote_type_conference  \
congress session chamber rollnumber icpsr                          
101      2.0     0       371        633.0                      0   
                                    1077.0                     0   
                                    1087.0                     0   
                                    2009.0                     0   
                                    2605.0                     0   

                                            vote_type_pass  \
congress session chamber rollnumber icpsr                    
101      2.0     0       371        633.0                0   
                                    1077.0               0   
                                    1087.0               0   
                                    2009.0               0   
                                    2605.0               0   

                                            vote_type_recommit  \
congress session chamber rollnumber icpsr                        
101      2.0     0       371        633.0                    0   
                                    1077.0                   0   
                                    1087.0                   0   
                                    2009.0                   0   
                                    2605.0                   0   

                                            vote_type_suspend  ...  \
congress session chamber rollnumber icpsr                      ...   
101      2.0     0       371        633.0                   0  ...   
                                    1077.0                  0  ...   
                                    1087.0                  0  ...   
                                    2009.0                  0  ...   
                                    2605.0                  0  ...   

                                            crs_policy_area_taxation  \
congress session chamber rollnumber icpsr                              
101      2.0     0       371        633.0                          0   
                                    1077.0   

In [ ]:
prior_votes = df.sort_index().groupby(["congress", "rollnumber", "icpsr"])["vote_for"].cumcount().to_frame("prior_votes_for_bill")
prior_votes.head()

congress  session  chamber  rollnumber  icpsr 
101       1.0      1        11          660.0     0
                                        1252.0    0
                                        1366.0    0
                                        4812.0    0
                                        9369.0    0
dtype: int64

In [12]:
df.join(prior_votes)

ValueError: Other Series must have a name